In [11]:
import torch
import numpy as np

In [1]:
EPS   = 0.001; # the regularization constant
sigma = 2**-16; # the kernel width
N     = 632;  # number of persons
d     = 100;  # number of features

In [2]:
N = 632
perm = torch.randperm(N)
print(torch.min(perm), torch.max(perm))

tensor(0) tensor(631)


In [3]:
idxtrain = perm[:N//2]
idxtest = perm[N//2:]
print(torch.min(idxtrain), torch.max(idxtrain))
print(torch.min(idxtest), torch.max(idxtest))

tensor(2) tensor(631)
tensor(0) tensor(628)


In [4]:
file_name = "viper//viper_features.npy"
mat = np.load(file_name, allow_pickle=True)
idxa = torch.from_numpy((mat.item().get('idxa')-1).astype('int64'))
idxb = torch.from_numpy((mat.item().get('idxb')-1).astype('int64'))

### first_ind is our set of similiaraity constraints so what we're doing is since idxtrain is a random permutation of 0:631 in 316 elements which act as indexes for the idxa thing and since we are talking 'bout Similaity pairwise constraint hence we're concatinating similiar element in pair e.g. let idxtrain = [[0, 345, 123]] be in R3 then idxa[0, idxtrain] = [x1, x2, x3] then first_ind = [[(x1, x2, x3), (x1, x2, x3)]] pairs. and since in D we want dissimmilar pairs hence we're randomly permutating them for the second phase of cat. e.g. second_ind =[idxb[[0, idxtrain]], idxb[[0, randperm(idxtrain)]]]

In [5]:
d = 100
X = torch.from_numpy(mat.item().get('ux'))[:d, :]

In [6]:
X.shape

torch.Size([100, 1264])

In [7]:
from demo import ret_logical, change_view
matches = ret_logical(N)

In [8]:
first_ind = change_view(torch.cat((idxa[0, idxtrain], idxa[0, idxtrain])))
second_ind = change_view(torch.cat((idxb[0, idxtrain], idxb[0, idxtrain[torch.randperm(N//2)]])))

In [9]:
S = torch.cat((first_ind[:, matches], second_ind[:, matches])) # must-link constraints
D = torch.cat((first_ind[:, torch.logical_not(matches)], second_ind[:, torch.logical_not(matches)])) # cannot-link constraints

In [10]:
S[0, 0], S[1, 0]

(tensor(442), tensor(1074))

In [13]:
from kernelmatrix import kernelmatrix
K = kernelmatrix(X, X2=torch.tensor([]), sigma=sigma)
print(K.shape)

torch.Size([1264, 1264])


In [23]:
from computeH import computeH
n = K.shape[0]
n1 = S.shape[1]
n0 = D.shape[1]
H0 = computeH(n, D[0, :], D[1, :])

In [27]:
torch.diagonal(H0)[:100]

tensor([0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
        1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
        0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 0., 0., 1., 1., 1., 1., 1.])